In [1]:
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from utils import *

In [2]:
# Parameters

image_size = (325, 325)
batch_size = 32

train_folder_path = r'C://Users//rolan//Desktop//Wooden_things//train'
train_csv_path = r'C://Users//rolan//Desktop//Wooden_things//train_csv.csv'

test_folder_path = r'C://Users//rolan//Desktop//Wooden_things//test'
test_csv_path = r'C://Users//rolan//Desktop//Wooden_things//test_csv.csv'

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

save_model_path = r'Models'
save_plot_path = r'Plots'

# DataSet

image_transforms = transforms.Compose([transforms.Resize(size = image_size), transforms.ToTensor()]) # Noramlisation if needed #transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))

train_data = CustomDataSet(folder_path = train_folder_path, csv_path = train_csv_path, transforms = image_transforms)
test_data = CustomDataSet(folder_path = test_folder_path, csv_path = test_csv_path, transforms = image_transforms)

# DataLoaders
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)

cuda


In [3]:
# Sample batch
#plot_batch_samples(data_loader=train_loader, batch_size=batch_size)

In [4]:
# Import model
from model_architecture import LinearWood

# Move to GPU
wt_model = LinearWood().to(device=device)

optimizer = torch.optim.Adam(params = wt_model.parameters(), lr=0.001, weight_decay=0.001, betas = (0.85, 0.99))
#optimizer = torch.optim.SGD(params = wt_model.parameters(), lr=0.25, weight_decay=0.01)

loss = nn.CrossEntropyLoss()

In [5]:
# Train loop

def train(model, dataloader, loss_f, optim_f):
  # set the model to train mode?
  model.train()

  # for plotting loss with matplotlib
  loss_list = []

  for idx, x in enumerate(dataloader):

    # move inputs and labels to the GPU
    x, y = x['image'].to(device), x['label'].to(device)

    # forward pass
    prediction = model.forward(x)

    # calculate the loss
    loss = loss_f(prediction, y)

    # calculate the gradient
    loss.backward()

    # update weights with optimizer
    optim_f.step()

    # zero the gradients
    optim_f.zero_grad()

    #print the training loss
    if idx % 10 == 0:
      print(f'training loss = {loss.item()}')

    loss_list.append(loss.item())

  return loss_list

In [6]:
# Test loop

def test(model, dataloader, loss_f, dataset):
  # set the model to evaluate mode
  model.eval()
  loss_sum = 0
  accuracy_sum = 0

  #create loss and accuracy variables
  with torch.no_grad():

    # set a loop over the batches
    for x in dataloader:

      # move inputs and labels to the GPU #'image': self.image, 'label'
      x, y = x['image'].to(device), x['label'].to(device)

      # run the forwards pass
      prediction = model.forward(x)

      # calculate the loss
      loss = loss_f(prediction, y)

      # total loss over all the batches
      loss_sum += loss

      # toatl accuracy over all the batches
      accuracy_sum += (prediction.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

  # average loss
  avg_loss = loss_sum / len(dataloader)
  avg_loss = '{:f}'.format(avg_loss)

  #average accuracy
  avg_accuracy = accuracy_sum / len(dataset)

  # print the loss and accuracy
  print(f'Epoch loss = {avg_loss} \n Epoch accuracy = {avg_accuracy*100}%')

  return avg_loss, avg_accuracy*100

In [7]:
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from datetime import datetime

def train_loop(epochs):
    train_loss_list = []
    test_loss_list = []
    accuracy_list= []
    current_time = str(datetime.now()).replace(' ', '').replace('.', '').replace(':', '')

    for idx, i in enumerate(range(epochs)):
        print(f'Epoch{i+1} \n --------------------------------------')

        #train the model
        train_loss = train(model = wt_model, loss_f=loss, optim_f=optimizer, dataloader=train_loader)
        test_loss, accuracy = test(model = wt_model, loss_f = loss, dataloader = test_loader, dataset = test_data)

        '''plot the graphs as the model trains'''
        #sort the data for the plots        
        for i in train_loss:
            train_loss_list.append(round(i, 3))
        test_loss_list.append(round(float(test_loss), 3))
        accuracy_list.append(accuracy)

        #close previous plots to free up memory
        plt.close('all')
        
        #create and format the figure
        loss_fig, (ax_train, ax_test, ax_acc) = plt.subplots(nrows=3, ncols=1)
        loss_fig.set_size_inches(5, 5)
        loss_fig.set_dpi(72)
        plt.subplots_adjust(hspace=1, wspace=1)

        #plot the data on the axes
        ax_train.plot(list(range(len(train_loader)*(idx+1))), train_loss_list, color='#1f77b4')
        ax_train.set_title('Training Loss/batch')
        ax_train.set_xlabel('Batches')
        ax_train.set_ylabel('Loss')
        ax_train.grid(True)

        ax_test.plot(list(range(idx+1)), test_loss_list, color='#ff7f0e')
        ax_test.set_title('Test Loss/Epoch')
        ax_test.set_xlabel('Epochs')
        ax_test.set_ylabel('Loss')
        ax_test.grid(True)

        ax_acc.plot(list(range(idx+1)), accuracy_list, color='#2ca02c')
        ax_acc.set_title('Model Accuracy')
        ax_acc.set_xlabel('Epochs')
        ax_acc.set_ylabel('Accuracy')
        ax_acc.grid(True)

        #save the model
        a = 0
        if idx % 10 == 0:
            a += 1
            torch.save(obj = wt_model.state_dict(), f=f'{save_model_path}//Experiment_{a}_InstanceNorm2D_{current_time[:-12]}_{str(accuracy)[:2]}.pth')
            print(f'Saved model as Experiment_{a}.pth')
        print('Finished')

        #plt.pause(0.1)
        plt.show()

        loss_fig.savefig(fname=f'{save_plot_path}//model_x_{current_time}.pdf', dpi=300)

In [8]:
train_loop(epochs = 81)

Epoch1 
 --------------------------------------


ValueError: Expected more than 1 spatial element when training, got input size torch.Size([32, 512, 1, 1])